In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("version2.csv")
df.head()

,Unnamed: 0,company,company_public_response,company_response,complaint_id,narratives,consumer_consent_provided,consumer_disputed,date_received,date_sent_to_company,...,state,sub_issue,sub_product,submitted_via,tags,timely,zip_code,Qtr,product_new,label
0,0,Experian Information Solutions Inc.,Company has responded to the consumer and the ...,Closed with explanation,2788944,There are many mistakes appear in my report wi...,Consent provided,NaN,2018-01-20,2018-01-20,...,MD,Account information incorrect,Credit reporting,Web,NaN,Yes,212XX,2018Q1,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
1,1,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Company has responded to the consumer and the ...,Closed with explanation,2773990,I was pulling my credit to looking into buying...,Consent provided,NaN,2018-01-06,2018-01-06,...,FL,Information belongs to someone else,Credit reporting,Web,NaN,Yes,347XX,2018Q1,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
2,2,"Caliber Home Loans, Inc.",NaN,Closed with explanation,2737995,"Since XXXX XXXX, I was working with a loan con...",Consent provided,NaN,2017-11-26,2017-11-26,...,MD,NaN,Conventional home mortgage,Web,NaN,Yes,208XX,2017Q4,Mortgage,"('Mortgage', 'apply for mortgage')"
3,3,"Select Management Resources, LLC",Company believes it acted appropriately as aut...,Closed with non-monetary relief,2674806,On XX/XX/XXXX I went to NTL in Delaware for a ...,Consent provided,NaN,2017-09-15,2017-09-15,...,PA,NaN,Title loan,Web,NaN,Yes,190XX,2017Q3,Loan,"('loan', 'payment')"
4,4,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Company has responded to the consumer and the ...,Closed with non-monetary relief,2683460,"Hello CFPB, I am sending this complaint to you...",Consent provided,NaN,2017-09-25,2017-09-25,...,MD,Their investigation did not fix an error on yo...,Credit reporting,Web,NaN,Yes,212XX,2017Q3,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."


In [3]:
df.shape

(87592, 22)

In [4]:
text = df['narratives'].tolist()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
import nltk, re, string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stop_words = stopwords.words('english')
words = set(nltk.corpus.words.words())

def get_doc_tokens(doc):
    lower = " ".join([w for w in doc.split() if len(w)>1]).lower()
    tokens = nltk.word_tokenize(lower)

    #Extract all tokens which are not in stop_words and punctuation
    my_tokens = [token.strip() for token in tokens\
             if token.strip() not in stop_words\
             and token.strip() not in string.punctuation]
    
    #Remove non-English words
    my_tokens1=[token for token in my_tokens if token in words]
    
  
    #Stemming all tokens
    porter_stemmer = PorterStemmer()
    stem_tokens = [porter_stemmer.stem(my_token) for my_token in my_tokens1]

      #get the token_count
    token_count = nltk.FreqDist(stem_tokens)
    
    return token_count

In [8]:
tfidf_vect = TfidfVectorizer(tokenizer = get_doc_tokens)
dtm = tfidf_vect.fit_transform(text)

In [9]:
dtm.shape

(87592, 11299)

In [10]:
label = df['label'].tolist()
len(label)

87592

In [11]:
all_label = list(set(label))

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(dtm, label,\
                                    test_size = 0.3, stratify = label,\
                                                    random_state = 123)

In [13]:
from sklearn.model_selection import cross_validate
from sklearn import svm 

metrics = ['precision_macro','recall_macro','f1_macro']
clf = svm.LinearSVC().fit(X_train, y_train)
predicted = clf.predict(X_test)
precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, labels = all_label)
print(classification_report(y_test, predicted, target_names = all_label))

                                                                                                                                                                      precision    recall  f1-score   support

                                                                                                            ('Credit card or prepaid card', 'Getting a credit card')       0.31      0.21      0.25       198
                                                                                                                           ('Credit card or prepaid card', 'others')       0.45      0.56      0.50       848
                                                                                                                    ('Credit card or prepaid card', 'close account')       0.50      0.38      0.43       181
                                                                                                                              ('loan', 'Managing the loan or lease')       0.43

In [ ]:
new_doc = ["I order XXXX internet. They installed internet and a phone line. I told them I don't need a phone line and never ordered one. They said they will removed the phone service which they did but a year later I got collection bill from I C SYSTEM INC stating I owe XXXX. I contacted XXXX about it and was promised they will notify I C SYSTEM INC to stop notification. Notification of collection did stop but it still show up on my credit. every time I called the promise it will be removed but it's still there affecting my credit score"]
X_new_tfidf = tfidf_vect.transform(new_doc)
pred_new = clf.predict(X_new_tfidf)
pred_new

array(["('Debt collection', 'Attempts to collect debt not owed')"], 
      dtype='<U164')

In [ ]:
new_doc = list(str(input("input: ")))
X_new_tfidf = tfidf_vect.transform(new_doc)
X_new_tfidf.shape
pred_new = clf.predict(X_new_tfidf)
pred_new

### Grid Search

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

text_clf = Pipeline([('tfidf',TfidfVectorizer()),
                    ('clf',svm.LinearSVC())])

parameters = {'tfidf__min_df':[10, 50],
             'tfidf__stop_words':[None,"english"],
             "clf__C":[1, 10, 100, 1000]}
gs_clf = GridSearchCV(text_clf, param_grid = parameters,\
                     scoring = "f1_macro", cv = 5)
gs_clf = gs_clf.fit(text,label)
for param_name in gs_clf.best_params_:
    print(param_name,": ", gs_clf.best_params_[param_name])
print("best f1 score: ", gs_clf.best_score_)
warnings.filterwarnings('ignore')

In [ ]:
df[df['label']=="('Checking or savings account', 'Managing an account')"]['narratives'].tolist()[3] 

### Cross Validation

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate

clf2 = svm.LinearSVC()
cv = cross_validate(clf2,dtm,label,scoring = metrics, cv = 5)
cv['test_f1_macro']

In [ ]:
np.mean(cv['test_f1_macro'])

### TruncatedSVD 

In [ ]:
#Try TruncatedSVD to reduce the dimension
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
svd = TruncatedSVD(n_components = 5000).fit(dtm.T)

In [ ]:
#Display the reduced dataset
svd_df = pd.DataFrame(svd.components_.T)
svd_df